## 准备数据

In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [12]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(shape=[28*28,100],dtype=tf.float32,
                              initial_value=tf.random.uniform(
                                shape=[28*28,100],minval=-0.1,maxval=0.1
                              ))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[100, 10],
                                                         minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        flat_x = tf.reshape(x,shape=[-1,28*28])
        y = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(y, self.W2) + self.b2
        return logits        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [13]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [15]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.0864038 ; accuracy 0.40961668
epoch 1 : loss 2.0821092 ; accuracy 0.41533333
epoch 2 : loss 2.077823 ; accuracy 0.42131665
epoch 3 : loss 2.0735447 ; accuracy 0.42716667
epoch 4 : loss 2.0692747 ; accuracy 0.43246666
epoch 5 : loss 2.0650125 ; accuracy 0.43783334
epoch 6 : loss 2.060758 ; accuracy 0.4434
epoch 7 : loss 2.0565112 ; accuracy 0.44893333
epoch 8 : loss 2.0522716 ; accuracy 0.45463333
epoch 9 : loss 2.0480394 ; accuracy 0.45945
epoch 10 : loss 2.043814 ; accuracy 0.46426666
epoch 11 : loss 2.0395958 ; accuracy 0.46863332
epoch 12 : loss 2.0353842 ; accuracy 0.47348332
epoch 13 : loss 2.0311794 ; accuracy 0.4789
epoch 14 : loss 2.0269809 ; accuracy 0.48363334
epoch 15 : loss 2.022789 ; accuracy 0.48851666
epoch 16 : loss 2.0186033 ; accuracy 0.49365
epoch 17 : loss 2.0144238 ; accuracy 0.49843332
epoch 18 : loss 2.0102506 ; accuracy 0.50311667
epoch 19 : loss 2.006083 ; accuracy 0.50811666
epoch 20 : loss 2.0019214 ; accuracy 0.51205
epoch 21 : loss 1.997765